# CNN for Regression from Noiseless Images to Labels

## Stage 2: Neural Architecture Search on a Subset of the Data (20k Samples)

In [1]:
import autokeras as ak
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import load_model, save_model

In [3]:
dataset = "../../data/data_v1_small.npz"

with np.load(dataset) as data:
    print("Available variables:", data.files)
    image_nonoise = data["img_nonoise"]
    label = data["label"]

Available variables: ['img', 'img_nonoise', 'label', 'psf_r', 'snr', 'sigma', 'train_test']


In [4]:
# Use unscaled noiseless images. Let Autokeras figure out if normalization is needed.
X = image_nonoise

# Scale the labels between -1 and 1 using the full dataset. We know the ranges of generated labels.
scaler = MinMaxScaler(feature_range=(-1, 1))
y = scaler.fit_transform(label)

In [6]:
# Reload the optimization state and finish training the Autokeras model
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

input_node = ak.ImageInput()
output_node = ak.ImageBlock(augment=False)(input_node)
output_node = ak.RegressionHead()(output_node)
auto = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=42, overwrite=False)
auto.fit(X, y, epochs=100, validation_split=0.2, verbose=2, callbacks=[early_stopping])

INFO:tensorflow:Reloading Oracle from existing project ./auto_model/oracle.json
INFO:tensorflow:Reloading Tuner from ./auto_model/tuner0.json
Train for 500 steps, validate for 125 steps
Epoch 1/100
500/500 - 766s - loss: 0.0414 - mean_squared_error: 0.0414 - val_loss: 0.0097 - val_mean_squared_error: 0.0097
Epoch 2/100
500/500 - 766s - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0048 - val_mean_squared_error: 0.0048
Epoch 3/100
500/500 - 766s - loss: 0.0042 - mean_squared_error: 0.0042 - val_loss: 0.0051 - val_mean_squared_error: 0.0051
Epoch 4/100
500/500 - 766s - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 5/100
500/500 - 766s - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0019 - val_mean_squared_error: 0.0019
Epoch 6/100
500/500 - 766s - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 7/100
500/500 - 766s - loss: 0.0018 - mean_squared_error: 0.0018 - val

Train for 500 steps, validate for 125 steps
Epoch 1/100
500/500 - 159s - loss: 0.0397 - mean_squared_error: 0.0397 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 2/100
500/500 - 157s - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.0043 - val_mean_squared_error: 0.0043
Epoch 3/100
500/500 - 157s - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0021 - val_mean_squared_error: 0.0021
Epoch 4/100
500/500 - 157s - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0018 - val_mean_squared_error: 0.0018
Epoch 5/100
500/500 - 157s - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 6/100
500/500 - 157s - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0028 - val_mean_squared_error: 0.0028
Epoch 7/100
500/500 - 157s - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0016 - val_mean_squared_error: 0.0016
Epoch 8/100
500/500 - 157s - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0015 - val

INFO:tensorflow:Oracle triggered exit
Train for 625 steps, validate for 125 steps
Epoch 1/100
625/625 - 195s - loss: 0.0335 - mean_squared_error: 0.0335 - val_loss: 0.0064 - val_mean_squared_error: 0.0064
Epoch 2/100
625/625 - 193s - loss: 0.0055 - mean_squared_error: 0.0055 - val_loss: 0.0036 - val_mean_squared_error: 0.0036
Epoch 3/100
625/625 - 193s - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0032 - val_mean_squared_error: 0.0032
Epoch 4/100
625/625 - 193s - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0022 - val_mean_squared_error: 0.0022
Epoch 5/100
625/625 - 193s - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 6/100
625/625 - 193s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 9.6465e-04 - val_mean_squared_error: 9.6465e-04
Epoch 7/100
625/625 - 193s - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0015 - val_mean_squared_error: 0.0015
Epoch 8/100
625/625 - 193s - loss: 0.0012 - mean_

In [9]:
# Show the best model. SeparableConv2D layers form Xception blocks.
model = auto.export_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 64, 64, 1)    3           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1664        normalization[0][0]              
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 32, 32, 256)  17216       conv2d[0][0]                     
______________________________________________________________________________________________

In [11]:
# Save the best model
save_model(model, 'noiseless_20k.tf', save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: noiseless_model.tf/assets
